Packages

In [10]:
import pandas as pd
import numpy as np
import os
import datetime

import matplotlib.pyplot as plt
from skimage import color, exposure
from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import accuracy_score

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [2]:
cd 'drive/My Drive/Colab Notebooks/notebooks'

/content/drive/My Drive/Colab Notebooks/notebooks


In [0]:
train = pd.read_pickle('../data/train.p')
test = pd.read_pickle('../data/test.p')
signnames = pd.read_csv('../data/signnames.csv')

In [0]:
xTrain, yTrain = train['features'], train['labels']
xTest, yTest = test['features'], test['labels']
labelsDict = signnames.to_dict()['b']

if yTrain.ndim == 1:
    yTrain = to_categorical(yTrain)
    
if yTest.ndim == 1:
    yTest = to_categorical(yTest)

In [0]:
inputShape = xTrain.shape[1:]
numClasses = yTrain.shape[1]

In [0]:
def trainModel(aModel: Sequential, aXTrain, aYTrain, aParamsFit = {}):
    aModel.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
    
    logdir = os.path.join('logs', datetime.datetime.now().strftime('%y%m%d-%H%M%S'))
    tensorboardCallback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq = 1)
    
    aModel.fit(
        aXTrain, 
        aYTrain,
        batch_size = aParamsFit.get('batch_size', 128),
        epochs = aParamsFit.get('epochs', 5),
        verbose = aParamsFit.get('verbose', 1),
        validation_data = aParamsFit.get('validation_data', (aXTrain, aYTrain)),
        callbacks = [tensorboardCallback])
    
    return model

def predict(aTrainedModel, aXTest, aYTest, scoring = accuracy_score):
  yTestNorm = np.argmax(aYTest, axis = 1)

  yPredProb = aTrainedModel.predict(aXTest)
  yPred = np.argmax(yPredProb, axis = 1)

  return scoring(yTestNorm, yPred)

In [0]:
def getCNN_v6(aInputShape: list, aNumClasses: int) -> Sequential:
    return Sequential([
        Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', input_shape = aInputShape),
        Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        MaxPool2D(),
        Dropout(0.3),
        
        Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'),
        MaxPool2D(),
        Dropout(0.3),

        Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'),
        MaxPool2D(),
        Dropout(0.3),
        
        Flatten(),

        Dense(1024, activation = 'relu'),
        Dropout(0.3),

        Dense(1024, activation = 'relu'),
        Dropout(0.3),

        Dense(aNumClasses, activation = 'softmax')
    ])

In [8]:
model = getCNN_v6(inputShape, numClasses)
modelTrained = trainModel(model, xTrain, yTrain)

predict(modelTrained, xTest, yTest)

Epoch 1/5
272/272 [==============================] - 5s 19ms/step - loss: 2.4734 - accuracy: 0.3280 - val_loss: 0.7177 - val_accuracy: 0.7797
Epoch 2/5
272/272 [==============================] - 5s 17ms/step - loss: 0.7525 - accuracy: 0.7636 - val_loss: 0.2169 - val_accuracy: 0.9448
Epoch 3/5
272/272 [==============================] - 5s 17ms/step - loss: 0.4076 - accuracy: 0.8736 - val_loss: 0.0918 - val_accuracy: 0.9759
Epoch 4/5
272/272 [==============================] - 5s 17ms/step - loss: 0.2632 - accuracy: 0.9189 - val_loss: 0.0674 - val_accuracy: 0.9804
Epoch 5/5
272/272 [==============================] - 5s 17ms/step - loss: 0.2217 - accuracy: 0.9339 - val_loss: 0.0508 - val_accuracy: 0.9876


0.9680272108843537

In [0]:
xTrainGray = color.rgb2gray(xTrain).reshape(-1, 32, 32, 1)
xTestGray = color.rgb2gray(xTest).reshape(-1, 32, 32, 1)

In [0]:
model = getCNN_v6((32, 32, 1), numClasses)
modelTrained = trainModel(model, xTrainGray, yTrain, aParamsFit = {})
predict(modelTrained, xTestGray, yTest)

Epoch 1/5
272/272 [==============================] - 7s 25ms/step - loss: 2.4870 - accuracy: 0.3061 - val_loss: 0.5920 - val_accuracy: 0.8092
Epoch 2/5
272/272 [==============================] - 6s 24ms/step - loss: 0.4874 - accuracy: 0.8417 - val_loss: 0.1375 - val_accuracy: 0.9563
Epoch 3/5
272/272 [==============================] - 6s 24ms/step - loss: 0.2286 - accuracy: 0.9271 - val_loss: 0.0614 - val_accuracy: 0.9830
Epoch 4/5
272/272 [==============================] - 6s 24ms/step - loss: 0.1495 - accuracy: 0.9525 - val_loss: 0.0343 - val_accuracy: 0.9902
Epoch 5/5
272/272 [==============================] - 6s 24ms/step - loss: 0.1182 - accuracy: 0.9626 - val_loss: 0.0204 - val_accuracy: 0.9941


0.9682539682539683

In [0]:
def getModel(aParams) -> Sequential:
    return Sequential([
        Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', input_shape = inputShape),
        Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        MaxPool2D(),
        Dropout(aParams['dropout_cnn_block_one']),
        
        Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'),
        MaxPool2D(),
        Dropout(aParams['dropout_cnn_block_two']),

        Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'),
        MaxPool2D(),
        Dropout(aParams['dropout_cnn_block_three']),
        
        Flatten(),

        Dense(1024, activation = 'relu'),
        Dropout(aParams['dropout_dense_block_one']),

        Dense(1024, activation = 'relu'),
        Dropout(aParams['dropout_dense_block_two']),

        Dense(numClasses, activation = 'softmax')
    ])

In [0]:
def funcObj(aParams):
  model = getModel(aParams)
  model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
    
  model.fit(
      xTrain, 
      yTrain,
      batch_size = int(aParams.get('batch_size', 128)),
      epochs = 5,
      verbose = 0
      )
  
  accuracy = model.evaluate(xTest, yTest)[1]
  print(aParams, f'accuracy = {accuracy}')
  
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [34]:
space = {
    'batch_size': hp.quniform('batch_size', 50, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0, 0.5),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0, 0.5)
}

best = fmin(funcObj, space, tpe.suggest, 20, Trials())

  1/138 [..............................]
 - ETA: 0s - loss: 0.8637 - accuracy: 0.5938

 16/138 [==>...........................]
 - ETA: 0s - loss: 0.1616 - accuracy: 0.9375

 27/138 [====>.........................]
 - ETA: 0s - loss: 0.7001 - accuracy: 0.9051

 41/138 [=======>......................]
 - ETA: 0s - loss: 0.4885 - accuracy: 0.9261

 55/138 [==========>...................]
 - ETA: 0s - loss: 0.5568 - accuracy: 0.8909

 68/138 [=============>................]
 - ETA: 0s - loss: 0.4786 - accuracy: 0.9035
